<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

<h1 style="text-align: center;">
    Линейная, LASSO, Ridge регрессии и их оптимизация
</h1>

Сегодня мы поработаем с оптимизацией в линейных алгоритмах (линейная регрессия и логистическая регрессия), а также разберём способы регуляризации данных методов. Для удобства данные методы будем реализовывать в виде классов в `Python` наподобие соотвествующих классов из `sklearn`.

In [ ]:
import numpy as np
import pandas as pd
import scipy.linalg as sla
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.linear_model import LinearRegression, Lasso, Ridge

%matplotlib inline

## 1. Линейная регрессия

Линейные методы предполагают, что между признаками объекта (features) и целевой переменной (target/label) существует линейная зависимость, то есть

$$
y = w_1 x_1 + w_2 x_2 + ... + w_k x_k + b,
$$

где $у$ - целевая переменная (то, что мы хотим предсказать), $x_i$ - признак объекта $x$, $w_i$ - вес $i$-го признака, $b$ - bias (смещение, свободный член).

Часто предполагают, что объект $x$ содержит в себе фиктивный признак равный $1$ для представления свободного члена $b$. В этом случае формула принимает простой вид:

$$
y = \langle w, x \rangle,
$$

где $\langle \cdot, \cdot \rangle$ - скалярное произведение векторов $w, x \in \mathbb{R}^n$.

В матричной форме, в случае, когда у нас есть $n$ объектов формулу можно переписать следующим образом:

$$
Y = Xw,
$$

где $Y$ - вектор-столбец размера $n$, $X$ - матрица признаков размера $n \times m$ (каждая строка матрицы есть описание признаков объекта), $w$ - вектор весов размера $m$.

**Лосс:**

$$
\begin{aligned}
L(y_{pred}, Y) &= \frac{1}{n}||y_{pred} - Y||^2_2 =\\
&= \frac{1}{n}||Xw - Y||^2_2 = \frac{1}{n}\sum_{i=1}^{n}\left(\sum_{j=1}^{m} x_{ij}w_j - y_i\right)^2 =: L(w)
\end{aligned}
$$

### 1.1 Аналитическое решение

Минимизация ошибки по методу наименьших квадратов даёт решение:

$$
w = (X^TX)^{-1}X^TY
$$

Реализуем класс линейной регрессии с помощью этой формулы:

In [ ]:
class MyLinearRegression:
    
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept

    def fit(self, X, y):
        # принимает на вход X, y и вычисляет веса по данной выборке
        # не забудьте про фиктивный признак, равный 1
        
        n, m = X.shape
        
        if self.fit_intercept:
            X_train = np.hstack((X, np.ones((n, 1))))
        else:
            x_train = X

        self.w = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y

        return self
        
    def predict(self, X):
        # принимает на вход X и возвращает ответы модели
        # не забудьте про фиктивный признак, равный 1

        n, m = X.shape

        if self.fit_intercept:
            X_test = np.hstack((X, np.ones((n, 1))))
        else:
            X_test = X
        
        y_pred = X_test @ self.w

        return y_pred
    
    def get_weights(self):
        return self.w

#### Тестирование

Сначала сгенерируем искусственные данные для теста моделей:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def linear_function(x):
    return 5 * x + 6

In [ ]:
# по признакам сгенерируем значения таргетов с некоторым шумом
objects_num = 50
X = np.linspace(-5, 5, objects_num)
y = linear_function(X) + np.random.randn(objects_num) * 5

# выделим половину объектов на тест
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)

Нанесём точки выборки и зависимость на график для наглядности:

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(X, linear_function(X), label='real', c='green')
plt.scatter(X_train, y_train, label='train', c='blue')
plt.scatter(X_test, y_test, label='test', c='orange')

plt.title("Generated dataset")
plt.grid(alpha=0.2)
plt.legend()
plt.show()

Обучим модель на `X_train` и предскажем результаты на `X_test`:

In [ ]:
regressor = MyLinearRegression()

regressor.fit(X_train[:, np.newaxis], y_train)

predictions = regressor.predict(X_test[:, np.newaxis])
w = regressor.get_weights()

In [ ]:
plt.figure(figsize=(20, 7))

ax = None

for i, types in enumerate([['train', 'test'], ['train'], ['test']]):
    ax = plt.subplot(1, 3, i + 1, sharey=ax)
    if 'train' in types:
        plt.scatter(X_train, y_train, label='train', c='blue')
    if 'test' in types:
        plt.scatter(X_test, y_test, label='test', c='orange')

    plt.plot(X, linear_function(X), label='real', c='green')
    plt.plot(X, regressor.predict(X[:, np.newaxis]), label='predicted', c='red')

    plt.xlabel('feature')
    plt.ylabel('target')
    plt.title(" ".join(types))
    plt.grid(alpha=0.2)
    plt.legend()

plt.show()

Сравним с реализацией из `sklearn`:

In [ ]:
sk_reg = LinearRegression().fit(X_train[:, np.newaxis], y_train)

plt.figure(figsize=(10, 7))
plt.plot(X, linear_function(X), label='real', c='green')

plt.scatter(X_train, y_train, label='train')
plt.scatter(X_test, y_test, label='test')
plt.plot(X, regressor.predict(X[:, np.newaxis]), label='ours', c='red', linestyle=':')
plt.plot(X, sk_reg.predict(X[:, np.newaxis]), label='sklearn', c='cyan', linestyle=':')

plt.title("Different Prediction")
plt.ylabel('target')
plt.xlabel('feature')
plt.grid(alpha=0.2)
plt.legend()
plt.show()

#### Результаты

In [ ]:
from sklearn.metrics import mean_squared_error

our_train_predictions = regressor.predict(X_train[:, np.newaxis])
our_test_predictions = regressor.predict(X_test[:, np.newaxis])

print('Our train MSE: ', mean_squared_error(y_train, our_train_predictions))
print('Our test MSE: ', mean_squared_error(y_test, our_test_predictions))

print()

sk_train_predictions = sk_reg.predict(X_train[:, np.newaxis])
sk_test_predictions = sk_reg.predict(X_test[:, np.newaxis])

print('sklearn train MSE: ', mean_squared_error(y_train, sk_train_predictions))
print('sklearn test MSE: ', mean_squared_error(y_test, sk_test_predictions))

### 1.2. Градиентная оптимизация

Обращение матрицы - очень тяжёлая операция. Кроме того, обратная матрица $(X^TX)^{-1}$ не всегда существует. По этим причинам мы воспользуемся методом градиентного спуска для оптимизации эмпирического риска.

Градиентый спуск заключается в:

1. Расчёте $\displaystyle\frac{\partial{L}}{\partial{w}}$ - градиента ошибки (значения целевой функции, то есть лосса $L(y_{pred}, y_{true})$) от значения параметров модели (весов $w$).

2. Шаге спуска - изменении весов $w$ в сторону антиградиента с некоторым коэффициентом $h$ (он же **learning rate**):

   $$
   w := w - h\frac{\partial{L}}{\partial{w}}
   $$

3. Повторении п.1 и п.2 пока не наблюдается сходимость (изменения ошибки малы или отсутсвуют).

Как правило, чаще используют стохастический градиентный спуск (**SGD**): выбирают случайный элемент обучающей выборки и изменяют коэффициенты модели по направлению антиградиента функции потерь на этом примере.

<img src="../img/linear_models_regularization_1.png" width=600>

**Цель:** реализовать новые классы линейной регрессии, в которой оптимизация проводится методами градиентного спуска.

**Предполагаемая зависимость:** $Y = Xw$, где $Y \in \mathbb{R}^{n\times 1}, X \in \mathbb{R}^{n\times m}, w \in \mathbb{R}^{m\times 1}$.

**Минимизируемая функция:**

$$
L(y_{pred}, Y) = \frac{1}{n} ||y_{pred} - Y||^2 = \frac{1}{n}||Xw - Y||^2 = \frac{1}{n}(Xw - Y)^T(Xw - Y) = \frac{1}{n}(w^TX^TXw - 2Y^TXw + Y^TY)
$$

**Градиент:**
$$
\frac{\partial{L}}{\partial{w}} = \frac{2}{n}(X^TXw - X^TY) = \frac{2}{n}X^T(y_{pred} - Y) \,\, \in \mathbb{R}^{m \times 1}
$$

In [ ]:
class MyGradientLinearRegression(MyLinearRegression):
    
    def __init__(self, **kwargs):
        # передаёт именные параметры родительскому конструктору
        super().__init__(**kwargs)
        self.w = None
    
    def fit(self, X, y, lr=0.01, max_iter=100):
        # принимает на вход X, y и вычисляет веса по данной выборке
        # не забудьте про фиктивный признак, равный 1

        n, k = X.shape

        # случайно инициализируем веса
        if self.w is None:
            self.w = np.random.randn(k + 1 if self.fit_intercept else k)
        
        if self.fit_intercept:
            X_train = np.hstack((X, np.ones((n, 1))))
        else:
            X_train = X

        self.losses = []
        
        for iter_num in range(max_iter):
            y_pred = self.predict(X)
            self.losses.append(mean_squared_error(y_pred, y))

            grad = self._calc_gradient(X_train, y, y_pred)

            assert grad.shape == self.w.shape, f'gradient shape {grad.shape} is not equal to weight shape {self.w.shape}'
            
            self.w -= lr * grad
            
        return self

    def _calc_gradient(self, X, y, y_pred):
        grad = 2 / len(X) * np.dot(X.T, y_pred - y)
        
        return grad
    
    def get_losses(self):
        return self.losses

#### Тестирование

Посчитаем предсказания на сгенерированном датасете:

In [ ]:
regressor = MyGradientLinearRegression(fit_intercept=True)

losses = regressor.fit(X_train[:, np.newaxis], y_train, max_iter=100).get_losses()

predictions = regressor.predict(X_test[:, np.newaxis])
w = regressor.get_weights()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(X, linear_function(X), label='real', c='green')

plt.scatter(X_train, y_train, label='train')
plt.scatter(X_test, y_test, label='test')
plt.plot(X, regressor.predict(X[:, np.newaxis]), label='predicted', c='red')

plt.grid(alpha=0.2)
plt.legend()
plt.show()

Построим также график лосса во время обучения:

In [ ]:
plt.figure(figsize=(10, 7))

plt.plot(losses)

plt.title('Gradient descent learning')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.ylim(bottom=0)
plt.grid(alpha=0.2)

plt.show()

### 1.3 SGD

Добавим к нашему градиентному спуску сэмплирование мини-батча, по которому будем считать градиент:

In [ ]:
class MySGDLinearRegression(MyGradientLinearRegression):
    
    def __init__(self, n_samples=10, **kwargs):
        # передаёт именные параметры родительскому конструктору
        super().__init__(**kwargs)
        self.n_samples = n_samples
        self.w = None

    def _calc_gradient(self, X, y, y_pred):
        # главное отличие в SGD - это использование подвыборки для шага оптимизации
        inds = np.random.choice(np.arange(X.shape[0]), size=min(self.n_samples, len(X)), replace=False)
        
        grad = 2 / len(inds) * np.dot(X[inds].T, y_pred[inds] - y[inds])

        return grad

#### Тестирование

Проведём аналогичный расчёт на сгенерированном датасете:

In [ ]:
regressor = MySGDLinearRegression(fit_intercept=True)

losses = regressor.fit(X_train[:, np.newaxis], y_train, max_iter=100).get_losses()

predictions = regressor.predict(X_test[:, np.newaxis])
w = regressor.get_weights()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(X, linear_function(X), label='real', c='green')

plt.scatter(X_train, y_train, label='train')
plt.scatter(X_test, y_test, label='test')
plt.plot(X, regressor.predict(X[:, np.newaxis]), label='predicted', c='red')

plt.grid(alpha=0.2)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))

plt.plot(losses)

plt.title('SGD learning')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.grid(alpha=0.2)
plt.show()

Протестируем раличные размеры батча для расчёта градиента:

In [ ]:
n_samples = [1, 2, 4]

In [ ]:
plt.figure(figsize=(10, 7))

for ns in n_samples:
    losses = MySGDLinearRegression(fit_intercept=True, n_samples=ns).fit(
        X_train[:, np.newaxis],
        y_train,
        lr=5e-3,
        max_iter=150,
    ).get_losses()
    plt.plot(losses, alpha=0.5, label=f'mini-batch size = {ns}')

plt.title('SGD learning')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.legend()
plt.ylim((0, 150))
plt.grid(alpha=0.2)
plt.show()

Как видно по графикам, размер подвыборки влияет на стабильность сходимости (чем меньше, тем больше и резче изменения весов).
При этом количество итераций для минимизации примерно одинаково.

**SGD** также обычно улучшают адаптивным уменьшением величины шага (подробнее в курсах про методы оптимизации и т.п.).

## 2. Логистическая регрессия

Задание аналогичное линейной регрессии, только мы переводим выходное значение в вероятность предсказания класса.

<img src='../img/linreg_regularization_1.png' width=600>

Для этого воспользуемся функцией $\displaystyle\sigma(x) = \frac{1}{1 + e^{-x}}$

<img src='../img/linreg_regularization_2.png' width=400>

Задача теперь формулируется так:

**Предсказания:**

$$
y_{pred}(x, w) = \frac{1}{1 + e^{-\langle x, w \rangle}}
$$

**Лосс (LogLoss):**

$$
L(w) = -y\, log\,y_{pred} - (1-y)\,log\,(1-y_{pred})
$$

<img src='../img/linreg_regularization_3.png' width=600>

**Градиент:**

$$
\frac{\partial{L}}{\partial{w}}
= \left(-\frac{y}{y_{pred}} + \frac{1-y}{1-y_{pred}}\right)\frac{\partial{y_{pred}}}{\partial{w}}
$$

$$
\frac{\partial{y_{pred}}}{\partial{w}} = \frac{1}{(1+e^{-\langle x, w \rangle})^2} e^{-\langle x, w \rangle} (-x) = y_{pred}(1-y_{pred})x
$$

$$
\frac{\partial{L}}{\partial{w}} = (y_{pred} - y) x
$$

In [ ]:
def logit(x, w):
    return np.dot(x, w)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class MyLogisticRegression(object):
    
    def __init__(self):
        self.w = None
    
    def fit(self, X, y, max_iter=100, lr=0.1):
        # принимает на вход X, y и вычисляет веса по данной выборке.
        # множество допустимых классов: {1, -1}
        # не забудьте про фиктивный признак равный 1!
        
        n, k = X.shape
        
        if self.w is None:
            self.w = np.random.randn(k + 1)
               
        losses = []
        
        X_train = np.concatenate((np.ones((n, 1)), X), axis=1)
        
        for iter_num in range(max_iter):
            y_pred = sigmoid(logit(X_train, self.w))
            grad = np.dot(X_train.T, y_pred - y) / len(X_train)

            self.w -= lr * grad

            losses.append(self.__loss(y, y_pred))
        
        return losses
        
    def predict_proba(self, X):
        # принимает на вход X и возвращает ответы модели
        n, k = X.shape
        X_ = np.concatenate((np.ones((n, 1)), X), axis=1)
        return sigmoid(logit(X_, self.w))

    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold
    
    def get_weights(self):
        return self.w
      
    def __loss(self, y, y_pred):
        y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)
        return np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=1000, centers=[[-2, 0.5], [3, -0.5]], cluster_std=1, random_state=42)

colors = ("red", "green")
colored_y = np.zeros(y.size, dtype=str)

for i, cl in enumerate([0, 1]):
    colored_y[y == cl] = str(colors[i])
    
plt.figure(figsize=(15, 10))
plt.scatter(X[:, 0], X[:, 1], c=colored_y)
plt.show()

In [ ]:
clf = MyLogisticRegression()

clf.fit(X, y, max_iter=1000)

w = clf.get_weights()

In [ ]:
from matplotlib.colors import ListedColormap

plt.figure(figsize=(15, 8))

eps = 0.1
xx, yy = np.meshgrid(np.linspace(np.min(X[:, 0]) - eps, np.max(X[:, 0]) + eps, 200),
                     np.linspace(np.min(X[:, 1]) - eps, np.max(X[:, 1]) + eps, 200))

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

Z = Z.reshape(xx.shape)

cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

plt.scatter(X[:, 0], X[:, 1], c=colored_y)
plt.show()

## 3. О регуляризации

Зачастую модель обучается на каких-то зашумленных данных. Веса модели после обучения подбираются для уменьшения ошибки целевой функции. На различных выборках модель может обучаться по-разному, но нам бы хотелось вычленять основную зависимость примерно одинаково, то есть **не переобучаться (overfit)** на данных. Иначе обучившись на одном сете, мы можем получать неожиданный результат на других данных.

То есть мы хотели бы штрафовать модель за ее сложность, чтобы выискивать более простые зависимости.

Одним словом нужно достигать некий bias/variance баланс для модели.

<img src='../img/linreg_regularization_4.png' width=600>

Для линейной модели дополнительные ограничения на веса выполняют роль регуляризации. Различают:
* $l_1$ регуляризацию (LASSO, least absolute shrinkage and selection operator), учитывание $||w||_1$

* $l_2$ регуляризацию (Ridge), учитывание $||w||^2_2$

* Elastic net - комбинация двух предыдущих

Далее напишем свои классы для Ridge и LASSO регрессий. Протестируем их на выборке из первого задания:

In [ ]:
objects_num = 50
X = np.linspace(-5, 5, objects_num)
y = linear_function(X) + np.random.randn(objects_num) * 5

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)

## 4. Ridge регрессия ($l_2$ регуляризация)

В Ridge мы штрафуем модель также на сумму квадратов всех её весов, таким образом:

**Лосс:** $L(w) = ||Xw - Y||^2_2 + \lambda||w||^2_2$, где $\lambda$ - гиперпараметр, отвечающий за степень регуляризации.

В привычном понимании:

**Лосс:** $L(w) = \sum_{i=1}^n\left(\sum_{j=1}^{m} x_{ij}w_j - y_i\right)^2 + \lambda\sum_{j=1}^{m}w_j^2$

Что стоит сказать про значения признаков? Они должны быть стандартизованы для одинаковых штрафов относительно друг друга (используется связка с `sklearn.preprocessing.StandardScaler`).

Кстати, наблюдается довольно интересная особенность Ridge, из-за того как мы определили функцию потерь:

In [ ]:
reg = Ridge(alpha=1).fit(np.hstack((X, X, X))[:, np.newaxis], np.hstack((y, y, y)))
np.append(reg.coef_, reg.intercept_)

In [ ]:
reg = Ridge(alpha=1/3).fit(X[:, np.newaxis], y)
np.append(reg.coef_, reg.intercept_)

### 4.1. Аналитическое решение

Из курса статистики известно, если:

$$
L(w) = ||Xw - Y||^2_2 + ||\Gamma w||^2_2
$$

минимизируется при:

$$
w = (X^TX + \Gamma^T \Gamma)^{-1}X^TY
$$

В нашем случае $\Gamma^T\Gamma = \lambda I$, если нет свободного члена, иначе:
$\Gamma^T\Gamma =
\left(\begin{matrix}
\lambda I & 0 \\
0 & 0
\end{matrix}\right)$

In [ ]:
class MyRidgeRegression(MyLinearRegression):
    
    def __init__(self, alpha=1.0, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha

    def fit(self, X, y):
        n, m = X.shape

        if self.fit_intercept:
            X_train = np.hstack((X, np.ones((n, 1))))
        else:
            X_train = X
        
        # найдём аналитическое решение
        lambda_I = self.alpha * np.eye(X_train.shape[1])
        
        if self.fit_intercept:
            lambda_I[-1, -1] = 0

        self.w = np.linalg.inv(X_train.T @ X_train + lambda_I) @ X_train.T @ y

        return self

Протестируем решение на нашем датасете:

In [ ]:
alpha = 1.0
regressor = MyRidgeRegression(alpha=alpha).fit(X_train[:, np.newaxis], y_train)

Проверим работу, сравнив с `linear_model.Ridge` из sklearn

In [ ]:
sklearn_reg = Ridge(alpha=alpha).fit(X_train[:, np.newaxis], y_train)
assert np.allclose(regressor.get_weights(), np.append(sklearn_reg.coef_, sklearn_reg.intercept_))
regressor.get_weights()

In [ ]:
plt.figure(figsize=(20, 7))

ax = None

for i, types in enumerate([['train', 'test'], ['train'], ['test']]):
    ax = plt.subplot(1, 3, i + 1, sharey=ax)
    if 'train' in types:
        plt.scatter(X_train, y_train, label='train', c='blue')
    if 'test' in types:
        plt.scatter(X_test, y_test, label='test', c='orange')

    plt.plot(X, linear_function(X), label='real', c='green')
    plt.plot(X, regressor.predict(X[:, np.newaxis]), label='predicted', c='red')

    plt.ylabel('target')
    plt.xlabel('feature')
    plt.title(" ".join(types))
    plt.grid(alpha=0.2)
    plt.legend()

plt.show()

### 4.2 SGD

Аналогично предыдущим заданиям нужно рассчитать значение градиента $\displaystyle\frac{\partial{L}}{\partial{w}}$.

**Имеем:**

$$
\begin{aligned}L(w) &= ||Xw - Y||^2_2 + ||\Gamma w||^2_2
= (Xw - Y)^T(Xw - Y) + w^T\Gamma^T\Gamma w = \\
&= w^TX^TXw - 2Y^TXw + Y^TY + w^T\Gamma^T\Gamma w =
w^T(X^TX + \Gamma^T\Gamma)w - 2Y^TXw + Y^TY
\end{aligned}
$$

**Градиент:**

$$
\frac{\partial{L}}{\partial{w}}
= 2(X^TX + \Gamma^T\Gamma)w - 2X^TY
$$

Будем также усреднять значения лосса по батчу данных (появится $\displaystyle\frac{1}{n_{sample}}$).

Реализуем наш класс Ridge регрессии используя стохастический градиентный спуск.

In [ ]:
class MySGDRidge(MySGDLinearRegression):
    def __init__(self, alpha=1.0, **kwargs):
        # передаёт именные параметры родительскому конструктору
        super().__init__(**kwargs)
        self.alpha = alpha
        self.w = None

    def _calc_gradient(self, X, y, y_pred):
        # главное отличие в SGD - это использование подвыборки для шага оптимизации
        inds = np.random.choice(np.arange(X.shape[0]), size=self.n_samples, replace=False)

        lambda_I = self.alpha * np.eye(self.w.shape[0])
        
        if self.fit_intercept:
            lambda_I[-1, -1] = 0

        grad = 2 * (X[inds].T @ X[inds] / self.n_samples + lambda_I) @ self.w
        grad -= 2 * X[inds].T @ y[inds] / self.n_samples

        return grad

In [ ]:
regressor = MySGDRidge(alpha=1.0, n_samples=20).fit(X[:, np.newaxis], y, max_iter=1000, lr=0.01)
l = regressor.get_losses()
regressor.get_weights()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(X, linear_function(X), label='real', c='green')

plt.scatter(X_train, y_train, label='train')
plt.scatter(X_test, y_test, label='test')
plt.plot(X, regressor.predict(X[:, np.newaxis]), label='predicted', c='red')

plt.grid(alpha=0.2)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))

plt.plot(l)

plt.title('Ridge learning with SGD')
plt.ylabel('loss')
plt.xlabel('iteration')
plt.grid(alpha=0.2)
plt.show()

## 5. LASSO регрессия ($l_1$ регуляризация)

В LASSO мы штрафуем модель также **на сумму модулей всех ее весов** (на $l_1$-норму весов), таким образом:

**Лосс:** $L(w) = ||Xw - Y||^2_2 + \lambda ||w||_1$, где $\lambda$ - гиперпараметр, отвечающий за степень регуляризации.

В привычном понимании:

**Лосс:** $L(w) = \sum_{i=1}^n\left(\sum_{j=1}^{m} x_{ij}w_j - y_i\right)^2 + \lambda\sum_{j=1}^{m}|w_j|$

Признаки опять же должны быть стандартизованы для одинаковых штрафов относительно друг друга!

В Lasso наблюдается поведение, которое нам бы и хотелось видеть:

In [ ]:
reg = Lasso(alpha=1.0).fit(np.hstack((X, X, X))[:, np.newaxis], np.hstack((y, y, y)))
np.append(reg.coef_, reg.intercept_)

In [ ]:
reg = Lasso(alpha=1.0).fit(X[:, np.newaxis], y)
np.append(reg.coef_, reg.intercept_)

### 5.1. Аналитическое решение (в общем случае не находится)

Изменение в типе нормы (с $l_2$ на $l_1$) может и выглядит достаточно просто, но при этом поведение регуляризации сильно меняется.

Решение находится достаточно просто если столбцы матрицы $X$ ортогональны (см. [stack exchange](https://stats.stackexchange.com/questions/17781/derivation-of-closed-form-lasso-solution)).

Также можно рассматривать эквивалентную задачу оптимизции (на что мы и будем ссылаться). Следующие задачи эквивалентны (из [теоремы Каруша-Куна-Таккера](https://ru.wikipedia.org/wiki/Условия_Каруша_—_Куна_—_Таккера)):

1. Для некоторого $c$:
$
\begin{cases}
\displaystyle L_2(w) \rightarrow \min_w \\
||w||_1 \leq c
\end{cases}
$

2. Для некоторого $\lambda$: $L(w) = L_2(w) + \lambda||w||_1 \rightarrow \displaystyle \min_w$

Первое представление задачи даёт нам геометрический смысл, как минимизация на гранях многомерного ромба.

<img src='../img/linreg_regularization_5.png' width=600>

### 5.2 SGD

В реализации `sklearn.linear_model.Lasso` используется так называемый **Stochastic Coordinat Descent (aka shooting algorithm)**, где веса пересчитываются по очереди.

<img src='../img/linreg_regularization_6.png' width=800>

Его мы реализовывать не будем.

Аналогично предыдущим заданиям для SGD нужно рассчитать значение градиента $\displaystyle\frac{\partial{L}}{\partial{w}}$.

**Лосс:**

$$
\begin{aligned}L(w) &= ||Xw - Y||^2_2 + ||\Gamma w||_1
= (Xw - Y)^T(Xw - Y) + ||\Gamma w||_1 = \\
&= w^TX^TXw - 2Y^TXw + Y^TY + \lambda\sum_{i=1}^{m-1}|w|
\end{aligned}
$$

**Градиент:**

$$
\frac{\partial{L}}{\partial{w}}
= 2(X^TXw - X^TY) + \lambda sign(w)
= 2X^T(y_{pred} - Y) + \lambda sign(w)
$$

где для приближения будем считать что $|\cdot|$ - дифференцируемая функция, ее производной является $sign(\cdot)$.

Реализуем наш класс Lasso регрессии используя стохастический градиентный спуск.

In [ ]:
def soft_sign(x, eps=1e-7):
    if abs(x) > eps:
        return np.sign(x)
    return x / eps


np_soft_sign = np.vectorize(soft_sign)

class MySGDLasso(MySGDLinearRegression):
    def __init__(self, alpha=1.0, **kwargs):
        # передаёт именные параметры родительскому конструктору
        super().__init__(**kwargs)
        self.alpha = alpha
        self.w = None

    def _calc_gradient(self, X, y, y_pred):
        # главное отличие в SGD - это использование подвыборки для шага оптимизации
        inds = np.random.choice(np.arange(X.shape[0]), size=self.n_samples, replace=False)

        sign_w = np_soft_sign(self.w)
        if self.fit_intercept:
            sign_w[-1] = 0

        grad = 2 / self.n_samples * X[inds].T @ (y_pred[inds] - y[inds])
        grad += self.alpha * sign_w
        
        return grad

Протестируем:

In [ ]:
regressor = MySGDLasso(alpha=1., n_samples=3).fit(X[:, np.newaxis], y, max_iter=1000, lr=0.01)
l = regressor.get_losses()
regressor.get_weights()

In [ ]:
sklearn_reg = Lasso().fit(X[:, np.newaxis], y)
np.append(sklearn_reg.coef_, sklearn_reg.intercept_)

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(X, linear_function(X), label='real', c='green')

plt.scatter(X_train, y_train, label='train')
plt.scatter(X_test, y_test, label='test')
plt.plot(X, regressor.predict(X[:, np.newaxis]), label='predicted', c='red')

plt.grid(alpha=0.2)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))

plt.plot(l)

plt.title('Lasso learning with SGD')
plt.ylabel('loss')
plt.xlabel('iteration')
plt.grid(alpha=0.2)
plt.show()

## Различия LASSO и Ridge

<img src='../img/linreg_regularization_7.png' width=600>

Между $l_1$ и $l_2$ регуляризациями существует несколько различий:

* **сложнее считать** из-за недифференцируемых углов шара в $l_1$-норме (в значении нуля для признаков)

* из-за отсутствия надлежащих поворотов с низким изменением лосса, появляется **зануление весов** для некоторых признаков

* **отсутствие аналитического решения** делает вычисления и теоретические значения весов при регуляризации более сложными

Протестируем методы на реальных данных:

In [ ]:
# !pip install -U scikit-learn

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
data = load_iris(as_frame=True).frame
names = data.columns
data.head()

In [ ]:
sns.pairplot(data, hue='target')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[names[:-1]], data[names[-1]])

In [ ]:
from sklearn.linear_model import SGDClassifier, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
# cls = make_pipeline(
#     StandardScaler(),
#     SGDClassifier(alpha=0.2, l1_ratio=0.1)
# )

cls = SGDClassifier(alpha=0.2, l1_ratio=0.1)

cls = cls.fit(X_train.to_numpy(), y_train)
preds_train = cls.predict(X_train)

accuracy_score(preds_train, y_train), f1_score(preds_train, y_train, average='macro')

In [ ]:
preds_test = cls.predict(X_test)

accuracy_score(preds_test, y_test), f1_score(preds_test, y_test, average='macro')

In [ ]:
# cls[1].coef_, cls[1].intercept_
cls.coef_, cls.intercept_

In [ ]:
from matplotlib.colors import ListedColormap

plt.figure(figsize=(15, 8))

eps = 0.1
xx, yy = np.meshgrid(np.linspace(np.min(X_train[names[0]]) - eps, np.max(X_train[names[0]]) + eps, 200),
                     np.linspace(np.min(X_train[names[1]]) - eps, np.max(X_train[names[1]]) + eps, 200))


cls.fit(X_train[names[:2]], y_train)
Z = cls.predict(np.c_[xx.ravel(), yy.ravel()])

Z = Z.reshape(xx.shape)

cmap_light = ListedColormap(['#AAAAFF', '#FFAAAA', '#AAFFAA'])
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

plt.scatter(X_train[names[0]], X_train[names[1]], c=y_train, cmap='brg')
plt.show()